In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [2]:
path = "/labs/lab07data/DO_record_per_line.json"
cl= spark.read.json(path)

In [4]:
cl.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)



In [18]:
cl.take(1)

[Row(cat=u'3/business_management|6/economics_finance', desc=u'This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They\u2019ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close accounts

In [51]:
targets = [26014, 14205, 12290, 17953, 12201, 21022]

In [201]:
cl[cl['id'].isin(targets)].select('lang', 'id').collect()

[Row(lang=u'ru', id=12201),
 Row(lang=u'es', id=12290),
 Row(lang=u'en', id=14205),
 Row(lang=u'es', id=17953),
 Row(lang=u'ru', id=21022),
 Row(lang=u'en', id=26014)]

In [61]:
for tr in targets:
    print cl[cl['id']==tr].take(1)[0][1]
    print ('#############################################################')

 The best way to prepare for CCIE Routing and Switching Version 5 certification. 
#############################################################
Highlights the important new features of the latest release of ColdFusion, a dynamic web application builder.
#############################################################

En este curso aprenderemos a modelar con 3Ds Max cuatro objetos imprescindibles para escenas de interiores, tales como sillas, cojines, sofas y mesas.
3dsMax es un programa de creación de gráficos y animación 3D desarrollado por Autodesk, en concreto la división Autodesk Media &amp; Entertainment (anteriormente Discreet). Creado inicialmente por el Grupo Yost para Autodesk, salió a la venta por primera vez en 1990 para DOS. 3ds Max, con su arquitectura basada en plugins, es uno de los programas de animación 3D más utilizado, especialmente para la creación de video juegos, anuncios de televisión, en arquitectura o en películas. Este curso es impartido por Juani Utrillas, gene

In [69]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

from pyspark.mllib.feature import Normalizer
normalizer_l2 = Normalizer(p=2)

import nltk
import re

In [115]:
tf = HashingTF(10000)
idf = IDF()

In [116]:
regex = re.compile(ur'[\w\d]{2,}', re.U)

def tokens(x):
    tokens = regex.findall(x.lower())
    return tokens    

In [210]:
def calc_cosine_similarity(a,b):
    return a.dot(b)

In [251]:
cl_ru = cl[cl['lang']=='ru']
cl_ru_rdd = cl[cl['lang']=='ru'].rdd
ru_12201 = cl_ru[cl_ru['id']==12201].rdd
ru_21022 = cl_ru[cl_ru['id']==21022].rdd

bag_ru = cl_ru_rdd.map(lambda x: x[1]).map(tokens)

bag_12201 = ru_12201.map(lambda x: x[1]).map(tokens)
bag_21022 = ru_21022.map(lambda x: x[1]).map(tokens)

tfs_ru = tf.transform(bag_ru)
tfs_12201 = tf.transform(bag_12201)
tfs_21022 = tf.transform(bag_21022)

idf_ru_model = idf.fit(tfs_ru)
tf_idf_ru = idf_ru_model.transform(tfs_ru)

tf_idf_12201 = idf_ru_model.transform(tfs_12201)
tf_idf_21022 = idf_ru_model.transform(tfs_21022)

norm_tf_idf_ru = normalizer_l2.transform(tf_idf_ru)

norm_tf_idf_12201 = normalizer_l2.transform(tf_idf_12201)
norm_tf_idf_21022 = normalizer_l2.transform(tf_idf_21022)

tr_12201 = norm_tf_idf_12201.take(1)[0]
tr_21022 = norm_tf_idf_21022.take(1)[0]

similarities_12201 = norm_tf_idf_ru.map(lambda x: calc_cosine_similarity(x, tr_12201))
similarities_21022 = norm_tf_idf_ru.map(lambda x: calc_cosine_similarity(x, tr_21022))

In [299]:
ru_index_list_12201 = []
ru_list_12201 = []
for e in list(similarities_12201.zipWithIndex().takeOrdered(11, lambda x: -x[0])):
    ru_index_list_12201.append(e[1])
ru_index_list_12201.pop(0)
for i in ru_index_list_12201:
    ru_list_12201.append(cl_ru_rdd.zipWithIndex().filter(lambda (value,index) : index == i). \
        map(lambda (value,index): value).take(1)[0][2])

In [300]:
ru_index_list_21022 = []
ru_list_21022 = []
for e in list(similarities_21022.zipWithIndex().takeOrdered(11, lambda x: -x[0])):
    ru_index_list_21022.append(e[1])
ru_index_list_21022.pop(0)
for i in ru_index_list_21022:
    ru_list_21022.append(cl_ru_rdd.zipWithIndex().filter(lambda (value,index) : index == i). \
        map(lambda (value,index): value).take(1)[0][2])

In [295]:
cl_en = cl[cl['lang']=='en']
cl_en_rdd = cl[cl['lang']=='en'].rdd
en_14205 = cl_en[cl_en['id']==14205].rdd
en_26014 = cl_en[cl_en['id']==26014].rdd

bag_en = cl_en_rdd.map(lambda x: x[1]).map(tokens)

bag_14205 = en_14205.map(lambda x: x[1]).map(tokens)
bag_26014 = en_26014.map(lambda x: x[1]).map(tokens)

tfs_en = tf.transform(bag_en)
tfs_14205 = tf.transform(bag_14205)
tfs_26014 = tf.transform(bag_26014)

idf_en_model = idf.fit(tfs_en)
tf_idf_en = idf_en_model.transform(tfs_en)

tf_idf_14205 = idf_en_model.transform(tfs_14205)
tf_idf_26014 = idf_en_model.transform(tfs_26014)

norm_tf_idf_en = normalizer_l2.transform(tf_idf_en)

norm_tf_idf_14205 = normalizer_l2.transform(tf_idf_14205)
norm_tf_idf_26014 = normalizer_l2.transform(tf_idf_26014)

tr_14205 = norm_tf_idf_14205.take(1)[0]
tr_26014 = norm_tf_idf_26014.take(1)[0]

similarities_14205 = norm_tf_idf_en.map(lambda x: calc_cosine_similarity(x, tr_14205))
similarities_26014 = norm_tf_idf_en.map(lambda x: calc_cosine_similarity(x, tr_26014))

In [297]:
en_index_list_14205 = []
en_list_14205 = []
for e in list(similarities_14205.zipWithIndex().takeOrdered(11, lambda x: -x[0])):
    en_index_list_14205.append(e[1])
en_index_list_14205.pop(0)
for i in en_index_list_14205:
    en_list_14205.append(cl_en_rdd.zipWithIndex().filter(lambda (value,index) : index == i). \
        map(lambda (value,index): value).take(1)[0][2])

In [298]:
en_index_list_26014 = []
en_list_26014 = []
for e in list(similarities_26014.zipWithIndex().takeOrdered(11, lambda x: -x[0])):
    en_index_list_26014.append(e[1])
en_index_list_26014.pop(0)
for i in en_index_list_26014:
    en_list_26014.append(cl_en_rdd.zipWithIndex().filter(lambda (value,index) : index == i). \
        map(lambda (value,index): value).take(1)[0][2])

In [255]:
cl_es = cl[cl['lang']=='es']
cl_es_rdd = cl[cl['lang']=='es'].rdd
es_12290 = cl_es[cl_es['id']==12290].rdd
es_17953 = cl_es[cl_es['id']==17953].rdd

bag_es = cl_es_rdd.map(lambda x: x[1]).map(tokens)

bag_12290 = es_12290.map(lambda x: x[1]).map(tokens)
bag_17953 = es_17953.map(lambda x: x[1]).map(tokens)

tfs_es = tf.transform(bag_es)
tfs_12290 = tf.transform(bag_12290)
tfs_17953 = tf.transform(bag_17953)

idf_es_model = idf.fit(tfs_es)
tf_idf_es = idf_es_model.transform(tfs_es)

tf_idf_12290 = idf_es_model.transform(tfs_12290)
tf_idf_17953 = idf_es_model.transform(tfs_17953)

norm_tf_idf_es = normalizer_l2.transform(tf_idf_es)

norm_tf_idf_12290 = normalizer_l2.transform(tf_idf_12290)
norm_tf_idf_17953 = normalizer_l2.transform(tf_idf_17953)

tr_12290 = norm_tf_idf_12290.take(1)[0]
tr_17953 = norm_tf_idf_17953.take(1)[0]

similarities_12290 = norm_tf_idf_es.map(lambda x: calc_cosine_similarity(x, tr_12290))
similarities_17953 = norm_tf_idf_es.map(lambda x: calc_cosine_similarity(x, tr_17953))

In [292]:
es_index_list_12290 = []
es_list_12290 = []
for e in list(similarities_12290.zipWithIndex().takeOrdered(11, lambda x: -x[0])):
    es_index_list_12290.append(e[1])
es_index_list_12290.pop(0)
for i in es_index_list_12290:
    es_list_12290.append(cl_es_rdd.zipWithIndex().filter(lambda (value,index) : index == i). \
        map(lambda (value,index): value).take(1)[0][2])

In [293]:
es_index_list_17953 = []
es_list_17953 = []
for e in list(similarities_17953.zipWithIndex().takeOrdered(11, lambda x: -x[0])):
    es_index_list_17953.append(e[1])
es_index_list_17953.pop(0)
for i in es_index_list_17953:
    es_list_17953.append(cl_es_rdd.zipWithIndex().filter(lambda (value,index) : index == i). \
        map(lambda (value,index): value).take(1)[0][2])

In [330]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter('ignore')

In [319]:
df = cl.toPandas()

In [325]:
def cos_m(x, y):
    return abs(cosine_similarity(x, y)[0][0])

In [331]:
df_es = pd.DataFrame()
df_es = df[df['lang'] == 'es']
df_es.reset_index(inplace = True, drop = True)

es_desc = list(df_es['desc'])
tf_v_es = TfidfVectorizer(norm=None, smooth_idf=False)
matrix_es = tf_v_es.fit_transform(es_desc).toarray()

cos_17953 = []

for i in range(matrix_es.shape[0]):
    
    cos_17953.append(cos_m(matrix_es[df_es[df_es['id'] == 17953].index[0]], matrix_es[i]))

df_es['cos_17953'] = cos_17953


df_es.sort_values(by = ['cos_17953', 'name', 'id'], ascending=[False, True, True], inplace = True)
top10_17953 = list(df_es['id'].iloc[1:11])

In [333]:
import json

with open("../lab07.json", "w") as outfile:
    json.dump({'12290':es_list_12290, '17953':top10_17953, '14205':en_list_14205, '26014':en_list_26014,
            '12201':ru_list_12201, '21022':ru_list_21022 }, outfile, indent=None)

In [334]:
with open("../lab07s.json", "w") as outfile:
    json.dump({'12290':es_list_12290, '17953':top10_17953, '14205':en_list_14205, '26014':en_list_26014,
            '12201':ru_list_12201, '21022':ru_list_21022 }, outfile, indent=None)